In [172]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [173]:
ratings = pd.read_csv('./data/ratings.dat', header=None, sep='::', engine='python',
                      names=('UserID','MovieID','Rating','Timestamp'))
ratings.shape

(1000209, 4)

In [174]:
ratings.isnull().sum()

UserID       0
MovieID      0
Rating       0
Timestamp    0
dtype: int64

In [175]:
users = pd.read_csv('./data/users.dat', header=None, sep='::', engine='python',
                      names=('UserID','Gender','Age','Occupation','Zip-code'))
users.head()

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [176]:
users.shape

(6040, 5)

In [177]:
movies = pd.read_csv('./data/movies.dat', header=None, sep='::', engine='python',
                      names=('MovieID','Title','Genres'))
movies.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


## Extracting Genres and creating their own columns

In [178]:
genres = ['Action','Adventure','Animation', "Children's", 'Comedy','Crime','Documentary','Drama','Fantasy',
          'Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western']

movies = pd.concat([movies,pd.DataFrame(columns=genres)])
movies.columns

Index([u'Action', u'Adventure', u'Animation', u'Children's', u'Comedy',
       u'Crime', u'Documentary', u'Drama', u'Fantasy', u'Film-Noir', u'Genres',
       u'Horror', u'MovieID', u'Musical', u'Mystery', u'Romance', u'Sci-Fi',
       u'Thriller', u'Title', u'War', u'Western'],
      dtype='object')

In [179]:
def add_genres(row):
    cur_genres = row['Genres'].split("|")
    for g in genres:
        if g in cur_genres:
            row[g] = 1
        else:
            row[g] = 0
    return row

In [180]:
movies = movies.apply(add_genres,axis=1)

In [181]:
movies.drop(['Genres'],axis=1,inplace=True)

In [182]:
## Merging movie and ratings dataframe through Inner-Join
movie_ratings = pd.merge(ratings,movies,on='MovieID',how='inner')

In [183]:
movie_ratings.head()

,UserID,MovieID,Rating,Timestamp,Action,Adventure,Animation,Children's,Comedy,Crime,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,Title,War,Western
0,1,1193,5,978300760,0,0,0,0,0,0,...,0,0,0,0,0,0,0,One Flew Over the Cuckoo's Nest (1975),0,0
1,2,1193,5,978298413,0,0,0,0,0,0,...,0,0,0,0,0,0,0,One Flew Over the Cuckoo's Nest (1975),0,0
2,12,1193,4,978220179,0,0,0,0,0,0,...,0,0,0,0,0,0,0,One Flew Over the Cuckoo's Nest (1975),0,0
3,15,1193,4,978199279,0,0,0,0,0,0,...,0,0,0,0,0,0,0,One Flew Over the Cuckoo's Nest (1975),0,0
4,17,1193,5,978158471,0,0,0,0,0,0,...,0,0,0,0,0,0,0,One Flew Over the Cuckoo's Nest (1975),0,0


In [184]:
movie_ratings.shape

(1000209, 23)

## Top 10 most viewed movies
**Assumption: Most Viewed is also a Most Reviewed Movie**

In [185]:
movie_ratings_count = movie_ratings.groupby('MovieID').size().reset_index(name='CountRatings')
movie_ratings_count = movie_ratings_count.sort_values('CountRatings',ascending=False)
movie_ratings_sorted = movie_ratings_count.merge(movies,on='MovieID',how='inner')[['MovieID', 'Title', 'CountRatings']]
movie_ratings_sorted.head(10)

,MovieID,Title,CountRatings
0,2858,American Beauty (1999),3428
1,260,Star Wars: Episode IV - A New Hope (1977),2991
2,1196,Star Wars: Episode V - The Empire Strikes Back...,2990
3,1210,Star Wars: Episode VI - Return of the Jedi (1983),2883
4,480,Jurassic Park (1993),2672
5,2028,Saving Private Ryan (1998),2653
6,589,Terminator 2: Judgment Day (1991),2649
7,2571,"Matrix, The (1999)",2590
8,1270,Back to the Future (1985),2583
9,593,"Silence of the Lambs, The (1991)",2578


## Top 20 rated movies at least rated by 40 users

In [186]:
movie_ratings_sorted[movie_ratings_sorted.CountRatings>=40].head(20)

,MovieID,Title,CountRatings
0,2858,American Beauty (1999),3428
1,260,Star Wars: Episode IV - A New Hope (1977),2991
2,1196,Star Wars: Episode V - The Empire Strikes Back...,2990
3,1210,Star Wars: Episode VI - Return of the Jedi (1983),2883
4,480,Jurassic Park (1993),2672
5,2028,Saving Private Ryan (1998),2653
6,589,Terminator 2: Judgment Day (1991),2649
7,2571,"Matrix, The (1999)",2590
8,1270,Back to the Future (1985),2583
9,593,"Silence of the Lambs, The (1991)",2578


## Genre Ranking for each Profession and Age Groups

In [413]:
genreRanking = pd.read_csv('GenreRankingWith_Age_Occupation.csv')
genreRanking

,Age,Occupation,Rank1,Rank2,Rank3,Rank4,Rank5
0,18-35,K-12 student,Action,Comedy,Drama,Sci-Fi,Thriller
1,36-50,K-12 student,Comedy,Drama,Romance,Action,Thriller
2,50+,K-12 student,Comedy,Drama,Action,Sci-Fi,Thriller
3,18-35,academic/educator,Drama,Comedy,Action,Thriller,Romance
4,36-50,academic/educator,Drama,Comedy,Action,Romance,Thriller
5,50+,academic/educator,Drama,Comedy,Action,Thriller,Romance
6,18-35,artist,Comedy,Drama,Action,Thriller,Sci-Fi
7,36-50,artist,Drama,Comedy,Action,Thriller,Romance
8,50+,artist,Drama,Comedy,Romance,Action,Thriller
9,18-35,clerical/admin,Comedy,Drama,Action,Thriller,Romance


### Process of getting above Result is down here.

In [187]:
user_ratings = pd.merge(users,movie_ratings,on='UserID',how='inner')
user_ratings.head()

,UserID,Gender,Age,Occupation,Zip-code,MovieID,Rating,Timestamp,Action,Adventure,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,Title,War,Western
0,1,F,1,10,48067,1193,5,978300760,0,0,...,0,0,0,0,0,0,0,One Flew Over the Cuckoo's Nest (1975),0,0
1,1,F,1,10,48067,661,3,978302109,0,0,...,0,0,1,0,0,0,0,James and the Giant Peach (1996),0,0
2,1,F,1,10,48067,914,3,978301968,0,0,...,0,0,1,0,1,0,0,My Fair Lady (1964),0,0
3,1,F,1,10,48067,3408,4,978300275,0,0,...,0,0,0,0,0,0,0,Erin Brockovich (2000),0,0
4,1,F,1,10,48067,2355,5,978824291,0,0,...,0,0,0,0,0,0,0,"Bug's Life, A (1998)",0,0


In [188]:
## mapping Occupation
occupation = {
            0:  "other",1:  "academic/educator",2:  "artist",3:  "clerical/admin",4:  "college/grad student",
              5:  "customer service",6:  "doctor/health care",7:  "executive/managerial",8:  "farmer",
              9:  "homemaker",10:  "K-12 student",11:  "lawyer",12:  "programmer",13:  "retired",
              14:  "sales/marketing",15:  "scientist",16:  "self-employed",17:  "technician/engineer",
              18:  "tradesman/craftsman",19:  "unemployed",20:  "writer"
             }
user_ratings.Occupation = user_ratings.Occupation.map(occupation)
user_ratings.head()

,UserID,Gender,Age,Occupation,Zip-code,MovieID,Rating,Timestamp,Action,Adventure,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,Title,War,Western
0,1,F,1,K-12 student,48067,1193,5,978300760,0,0,...,0,0,0,0,0,0,0,One Flew Over the Cuckoo's Nest (1975),0,0
1,1,F,1,K-12 student,48067,661,3,978302109,0,0,...,0,0,1,0,0,0,0,James and the Giant Peach (1996),0,0
2,1,F,1,K-12 student,48067,914,3,978301968,0,0,...,0,0,1,0,1,0,0,My Fair Lady (1964),0,0
3,1,F,1,K-12 student,48067,3408,4,978300275,0,0,...,0,0,0,0,0,0,0,Erin Brockovich (2000),0,0
4,1,F,1,K-12 student,48067,2355,5,978824291,0,0,...,0,0,0,0,0,0,0,"Bug's Life, A (1998)",0,0


In [189]:
## Dropping Users with age group less than 18.
## Re adjusting Age-Groups
user_ratings = user_ratings[user_ratings.Age!=1]
user_ratings.loc[user_ratings.Age==25,'Age'] = 18
user_ratings.loc[user_ratings.Age==45,'Age'] = 35
user_ratings.loc[user_ratings.Age>=50,'Age'] = 50
user_ratings.Age.value_counts()

18    579092
35    282636
50    111270
Name: Age, dtype: int64

In [323]:
## Getting Average Ratings of Users in a certain Age group and Occupation
mean_ratings = user_ratings.groupby(['Occupation','Age'])[genres].mean().reset_index()

In [324]:
mean_ratings.shape

(62, 20)

In [267]:
import operator
from collections import OrderedDict

In [401]:
## Creating a new Dataframe to store the fina; result table
cols = ['Occupation','Age','Rank1','Rank2','Rank3','Rank4','Rank5']
result_table = pd.DataFrame(columns=cols)

In [402]:
def sort_row(row):
    """Sorts by Average Rating and appends to a dataframe result_table"""
    row_d = row.to_dict()
    row_d = OrderedDict(sorted(row_d.items(),key=operator.itemgetter(1),reverse=True)[0:7])
    res = dict()
    i = 0
    items = row_d.items()
#     Changing Global variable from apply() is a bad idea, but had no choice.
    global result_table
    for c in cols:
        if(i<2):
            res[c] = [items[i][1]]
        else:
            res[c] = [items[i][0]]
        i+=1
    
    result_table = result_table.append(pd.DataFrame(res),)
    return row

In [403]:
## Applying sort_row function to rows in dataframe, the dataframe won't get changed
mean_ratings.apply(sort_row, axis=1)

,Occupation,Age,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,K-12 student,18,0.360167,0.215202,0.051636,0.104233,0.300192,0.073445,0.003528,0.291533,0.069596,0.012187,0.091084,0.036883,0.038486,0.119307,0.253368,0.224503,0.078576,0.019885
1,K-12 student,35,0.122517,0.072848,0.052980,0.066225,0.639073,0.059603,0.009934,0.374172,0.026490,0.013245,0.029801,0.033113,0.016556,0.165563,0.066225,0.089404,0.046358,0.009934
2,K-12 student,50,0.186275,0.083333,0.063725,0.034314,0.480392,0.083333,0.004902,0.352941,0.029412,0.014706,0.063725,0.024510,0.019608,0.137255,0.176471,0.151961,0.058824,0.000000
3,academic/educator,18,0.231338,0.119759,0.037152,0.062500,0.365865,0.080280,0.009968,0.389890,0.034262,0.016850,0.071243,0.037250,0.038573,0.170724,0.137882,0.180079,0.062133,0.015576
4,academic/educator,35,0.212554,0.124281,0.044386,0.078465,0.339695,0.065499,0.008445,0.416664,0.033481,0.021910,0.049041,0.055192,0.046215,0.165708,0.143133,0.163480,0.071583,0.022077
5,academic/educator,50,0.193952,0.108316,0.026243,0.046162,0.325967,0.075095,0.009450,0.442498,0.024280,0.036566,0.048270,0.046889,0.053867,0.168581,0.127581,0.168799,0.093777,0.029660
6,artist,18,0.252040,0.130306,0.046007,0.072742,0.370249,0.083601,0.009748,0.358470,0.038260,0.016701,0.080108,0.040991,0.035307,0.149008,0.155263,0.178536,0.059184,0.018098
7,artist,35,0.180632,0.102877,0.040912,0.061684,0.341544,0.072912,0.014877,0.410246,0.028561,0.025684,0.071439,0.064982,0.047158,0.165263,0.121544,0.170596,0.064211,0.020351
8,artist,50,0.179778,0.097411,0.030086,0.048335,0.326017,0.064858,0.007645,0.501356,0.036005,0.023921,0.031566,0.039211,0.056967,0.180271,0.115166,0.146732,0.068064,0.026634
9,clerical/admin,18,0.237030,0.116177,0.048263,0.077989,0.399633,0.081218,0.011022,0.349254,0.031619,0.014362,0.074148,0.044088,0.036573,0.157537,0.123803,0.185649,0.050824,0.015197


In [404]:
## Had no duplicates. Just for good measures.
result_table.drop_duplicates(subset=result_table.columns,inplace=True)

In [411]:
## renaming to the Age Groups
result_table.loc[result_table.Age==18,'Age'] = '18-35'
result_table.loc[result_table.Age==35,'Age'] = '36-50'
result_table.loc[result_table.Age==50,'Age'] = '50+'
## removing extra index column made by sort_row method
result_table = result_table.reset_index().drop(['index'],axis=1)
## Sorting Dataframe by Occupation and Age
result_table = result_table.sort_values(['Occupation','Age'])

In [412]:
## Saving Dataframe to csv file
result_table.to_csv('GenreRankingWith_Age_Occupation.csv',index=False)

## END